In [ ]:
# # Load Libraries
import glob
import os
import json
import numpy as np
import pandas as pd

# display all columns
pd.options.display.max_columns = 200

# # Get Multiple Json Files

json_files = []
chars = ['IRONCLAD','THE_SILENT','DEFECT','WATCHER'] # use same format as the names of your folders

for char in chars:
    #path = ".\{0}".format(char) # get files in the folder for sample. Use other to pull from where game saves.
    #path = "C:/Program Files (x86)/Steam/steamapps/common/SlayTheSpire/runs/{0}".format(char) # get files from where game saves
    path = os.path.join(os.getcwd(), char) #linux/windows
    print(path)
    json_file = glob.glob(os.path.join(path, "*.run")) #make list of paths linux/windows
    #json_file = glob.glob(os.path.join(path, "*.RUN")) #make list of paths
    json_files = json_files + json_file
    

# combine multiple json files into one data frame
prep_data = pd.DataFrame()

cnt = 0
tot_cnt = len(json_files)

for file in json_files:
    # get counts of all attempts (all runs)
    with open(file) as json_data:
        data = json.load(json_data)
    
    cnt = cnt + 1
    data = pd.json_normalize(data)
    
    prep_data = prep_data.append(data, ignore_index = True)
    print(str(cnt) + "/" + str(tot_cnt), sep = ' ', end = '\r', flush = True) # print dynamically
    
# filter runs
prep_data = prep_data[prep_data['ascension_level'] == 20] 
prep_data = prep_data[prep_data['floor_reached'] > 3]
prep_data = prep_data[prep_data['chose_seed'] == False]

# select the columns  to be used
prep_data = prep_data[['gold_per_floor', 'floor_reached', 'playtime', 'items_purged', 
       'play_id', 'local_time', 'campfire_choices', 'neow_cost', 'master_deck',
       'relics', 'potions_floor_usage', 'damage_taken', 'seed_played',
       'potions_obtained', 'path_per_floor', 'character_chosen',
       'items_purchased', 'campfire_rested', 'item_purchase_floors',
       'current_hp_per_floor', 'gold', 'neow_bonus',
       'chose_seed', 'campfire_upgraded', 'timestamp',
       'path_taken', 'build_version', 'purchased_purges', 'victory',
       'max_hp_per_floor', 'card_choices', 'relics_obtained', 'event_choices', 
       'boss_relics','items_purged_floors', 'potions_floor_spawned',
       'killed_by']]

# prep_data.to_json("LocalA20Runs.json")
# prep_data.to_csv("LocalA20Runs.csv", index = False)

# char_cnt = prep_data.groupby(['character_chosen', 'victory']).size()
# char_cnt

## Runs for Tableau
Main table to relate other tables to in tableau.

In [2]:
rft = prep_data[['play_id','local_time','character_chosen', 'floor_reached','neow_bonus','neow_cost','killed_by']]
rft = rft.reset_index()

In [3]:
#Convert local time to date
import datetime
from datetime import datetime

In [4]:
rft_index = 0
rft_df = pd.DataFrame()
for run in rft['local_time']:
    datetime_str = str(rft['local_time'][rft_index])
    datetime_obj = datetime.strptime(datetime_str,"%Y%m%d%H%M%S")
    date = datetime_obj.date()
    rft_row = {'run_id':rft['play_id'][rft_index],
               'run_date':date,
               'character_chosen':rft['character_chosen'][rft_index],
               'floor_reached':rft['floor_reached'][rft_index],
               'neow_bonus':rft['neow_bonus'][rft_index],
               'neow_cost':rft['neow_cost'][rft_index],
               'killed_by':rft['killed_by'][rft_index]
              }
    rft_row = pd.DataFrame([rft_row])
    rft_df = pd.concat([rft_df, rft_row], ignore_index=True)
    rft_index += 1
    
rft_df.to_csv("runsfortableau.csv", index = False) 
rft_df.head(5)

,run_id,run_date,character_chosen,floor_reached,neow_bonus,neow_cost,killed_by
0,02be86b6-8687-4d42-9102-c10b61a7b245,2022-05-04,IRONCLAD,57,THREE_ENEMY_KILL,NONE,NaN
1,8a36d361-b285-442a-b60f-e7ac22e0598d,2022-05-04,IRONCLAD,11,ONE_RARE_RELIC,TEN_PERCENT_HP_LOSS,Looter
2,c499fd7e-f334-4232-a8ff-e6a41031c2ba,2022-05-06,IRONCLAD,57,TWO_FIFTY_GOLD,CURSE,NaN
3,c8d23175-f597-4d40-aeea-b1dc82af0a29,2022-05-11,IRONCLAD,57,TWO_FIFTY_GOLD,CURSE,NaN
4,db375d67-e0fa-41a0-ad78-b7d5eafa1dfa,2022-05-15,IRONCLAD,16,TWENTY_PERCENT_HP_BONUS,PERCENT_DAMAGE,Hexaghost


## Boss Relics output for Tableau

In [5]:
br_ps = prep_data[['play_id','character_chosen','boss_relics','victory','floor_reached']] #br_ps = boss_relic picked/skipped
br_ps = br_ps.reset_index()

In [6]:
#Boss Relics Output for Tableau
index_to_use = -1
df1= pd.DataFrame()
var = (len(br_ps['play_id'])-1)

for run in br_ps['boss_relics']:
    acts = 0
    index_to_use += 1
    for act in run:
        acts += 1
        for relic in act['not_picked']:
            br_ps1={
                "play_Id":br_ps['play_id'][index_to_use],
                'character_chosen':br_ps['character_chosen'][index_to_use],
                'boss_relic': relic,
                'br_picked_skipped':'Skipped',
                'victory':br_ps['victory'][index_to_use],
                'floor_reached':br_ps['floor_reached'][index_to_use],
                'Act': acts
                }
            df_dictionary = pd.DataFrame([br_ps1])
            df1 = pd.concat([df1, df_dictionary], ignore_index=True)
        br_ps2={
            "play_Id":br_ps['play_id'][index_to_use],
            'character_chosen':br_ps['character_chosen'][index_to_use],
            'boss_relic': act.get('picked'),
            'br_picked_skipped':'Picked',
            'victory':br_ps['victory'][index_to_use],
            'floor_reached':br_ps['floor_reached'][index_to_use],
            'Act': acts
            }
        df_dictionary = pd.DataFrame([br_ps2])
        df1 = pd.concat([df1, df_dictionary], ignore_index=True) 

df1['victory'] = df1['victory'].astype(int)
df1.to_csv("Boss Relics Tableau.csv", index = False)                 
df1.head(5)  

,play_Id,character_chosen,boss_relic,br_picked_skipped,victory,floor_reached,Act
0,02be86b6-8687-4d42-9102-c10b61a7b245,IRONCLAD,Ectoplasm,Skipped,1,57,1
1,02be86b6-8687-4d42-9102-c10b61a7b245,IRONCLAD,Black Blood,Skipped,1,57,1
2,02be86b6-8687-4d42-9102-c10b61a7b245,IRONCLAD,Black Star,Picked,1,57,1
3,02be86b6-8687-4d42-9102-c10b61a7b245,IRONCLAD,Busted Crown,Skipped,1,57,2
4,02be86b6-8687-4d42-9102-c10b61a7b245,IRONCLAD,Sozu,Skipped,1,57,2


In [7]:
#print version to understand above
index_to_use = -1

var = (len(br_ps['play_id'])-1)

for run in br_ps['boss_relics'][:3]:
    if index_to_use == var:
        break
    acts = 0
    index_to_use += 1
    print('-----------------------------------------------------------------------------------------------------------------------------------------------------')
    print(index_to_use)
    for act in run:
        acts += 1
        for relic in act['not_picked']:
            print(f"Play_Id:{br_ps['play_id'][index_to_use]}, character_chosen:{br_ps['character_chosen'][index_to_use]}, Boss Relic: {relic},  Skipped, victory:{br_ps['victory'][index_to_use]}, floor_reached:{br_ps['floor_reached'][index_to_use]}, Act: {acts}")
        print(f"Play_Id:{br_ps['play_id'][index_to_use]}, character_chosen:{br_ps['character_chosen'][index_to_use]}, Boss Relic: {act.get('picked')},  Picked, victory:{br_ps['victory'][index_to_use]}, floor_reached:{br_ps['floor_reached'][index_to_use]}, Act: {acts}") 

-----------------------------------------------------------------------------------------------------------------------------------------------------
0
Play_Id:02be86b6-8687-4d42-9102-c10b61a7b245, character_chosen:IRONCLAD, Boss Relic: Ectoplasm,  Skipped, victory:True, floor_reached:57, Act: 1
Play_Id:02be86b6-8687-4d42-9102-c10b61a7b245, character_chosen:IRONCLAD, Boss Relic: Black Blood,  Skipped, victory:True, floor_reached:57, Act: 1
Play_Id:02be86b6-8687-4d42-9102-c10b61a7b245, character_chosen:IRONCLAD, Boss Relic: Black Star,  Picked, victory:True, floor_reached:57, Act: 1
Play_Id:02be86b6-8687-4d42-9102-c10b61a7b245, character_chosen:IRONCLAD, Boss Relic: Busted Crown,  Skipped, victory:True, floor_reached:57, Act: 2
Play_Id:02be86b6-8687-4d42-9102-c10b61a7b245, character_chosen:IRONCLAD, Boss Relic: Sozu,  Skipped, victory:True, floor_reached:57, Act: 2
Play_Id:02be86b6-8687-4d42-9102-c10b61a7b245, character_chosen:IRONCLAD, Boss Relic: SlaversCollar,  Picked, victory:True, 

## Enemy Damage Taken Tableau

In [8]:
edt = prep_data[['play_id','damage_taken']]
edt = edt.reset_index()
edt.head(5)

,index,play_id,damage_taken
0,0,02be86b6-8687-4d42-9102-c10b61a7b245,"[{'damage': 0.0, 'enemies': '2 Louse', 'floor'..."
1,1,8a36d361-b285-442a-b60f-e7ac22e0598d,"[{'damage': 10.0, 'enemies': 'Small Slimes', '..."
2,2,c499fd7e-f334-4232-a8ff-e6a41031c2ba,"[{'damage': 6, 'enemies': '2 Louse', 'floor': ..."
3,3,c8d23175-f597-4d40-aeea-b1dc82af0a29,"[{'damage': 4, 'enemies': 'Small Slimes', 'flo..."
4,4,db375d67-e0fa-41a0-ad78-b7d5eafa1dfa,"[{'damage': 0, 'enemies': '2 Louse', 'floor': ..."


In [9]:
print(edt['damage_taken'][1])

[{'damage': 10.0, 'enemies': 'Small Slimes', 'floor': 1.0, 'turns': 4.0}, {'damage': 14.0, 'enemies': 'Jaw Worm', 'floor': 4.0, 'turns': 4.0}, {'damage': 7.0, 'enemies': 'Cultist', 'floor': 7.0, 'turns': 4.0}, {'damage': 17.0, 'enemies': 'Lagavulin', 'floor': 8.0, 'turns': 7.0}, {'damage': 31, 'enemies': 'Gremlin Nob', 'floor': 10, 'turns': 4}, {'damage': 11, 'enemies': 'Looter', 'floor': 11, 'turns': 1}]


In [10]:
dt_df = pd.DataFrame()
dt_index = 0
dt_var =(len(edt['play_id']))-1
for enemy in edt['damage_taken']:
    if dt_index == dt_var:
        break
    for x in enemy:
        dt_row = {
            'play_id':edt['play_id'][dt_index],
            'enemy':x.get('enemies'),
            'damage_taken':x.get('damage'),
            'floor':x.get('floor'),
            'turns':x.get('turns')
            }
        dt_row = pd.DataFrame([dt_row])
        dt_df = pd.concat([dt_df, dt_row], ignore_index=True)
    dt_index +=1
    
dt_df[['damage_taken','floor','turns']] = dt_df[['damage_taken','floor','turns']].astype(int)
dt_df.to_csv("Damage Taken.csv", index = False) 
dt_df

,play_id,enemy,damage_taken,floor,turns
0,02be86b6-8687-4d42-9102-c10b61a7b245,2 Louse,0,1,1
1,02be86b6-8687-4d42-9102-c10b61a7b245,Cultist,0,5,2
2,02be86b6-8687-4d42-9102-c10b61a7b245,Gremlin Nob,0,6,1
3,02be86b6-8687-4d42-9102-c10b61a7b245,Lagavulin,16,8,7
4,02be86b6-8687-4d42-9102-c10b61a7b245,Small Slimes,0,11,1
...,...,...,...,...,...
1511,0f018942-fd29-4052-8572-940cbc3501fe,3 Darklings,0,48,3
1512,0f018942-fd29-4052-8572-940cbc3501fe,Donu and Deca,13,50,4
1513,0f018942-fd29-4052-8572-940cbc3501fe,Time Eater,0,51,3
1514,0f018942-fd29-4052-8572-940cbc3501fe,Shield and Spear,0,55,1


## Cards for Tableau

In [11]:
#output play_id, character_chosen, Card_Name, floor, victory, Card_Picked_Skipped

cft = prep_data[['play_id','character_chosen','card_choices','victory']]
cft = cft.reset_index()
cft['card_choices'][0]

[{'not_picked': ['Searing Blow', 'Perfected Strike'],
  'picked': 'Whirlwind',
  'floor': 1.0},
 {'not_picked': ['True Grit', 'Seeing Red'], 'picked': 'Anger', 'floor': 5},
 {'not_picked': ['Intimidate', 'Uppercut'], 'picked': 'Disarm', 'floor': 6},
 {'not_picked': ['Headbutt', 'Clash'], 'picked': 'Shrug It Off', 'floor': 8},
 {'not_picked': ['Headbutt', 'Fire Breathing', 'Searing Blow'],
  'picked': 'SKIP',
  'floor': 11},
 {'not_picked': ['Inflame', 'Warcry'], 'picked': 'Iron Wave', 'floor': 13},
 {'not_picked': ['Evolve', 'Dropkick'], 'picked': 'Fiend Fire', 'floor': 14},
 {'not_picked': ['Immolate', 'Juggernaut'],
  'picked': 'Impervious',
  'floor': 16},
 {'not_picked': ['Clothesline', 'Burning Pact', 'Infernal Blade'],
  'picked': 'SKIP',
  'floor': 18},
 {'not_picked': ['Cleave',
   'Dropkick',
   'Flex',
   'Seeing Red',
   'Shrug It Off',
   'Clash',
   'Warcry',
   'Body Slam',
   'Thunderclap',
   'Armaments',
   'Feel No Pain',
   'Infernal Blade',
   'True Grit',
   'Heavy

In [12]:
cft_index = 0
cft_df = pd.DataFrame()
cft_var =(len(cft['play_id']))-1
for floor in cft['card_choices']:
    if cft_index == cft_var:
        break
    for choice in floor:   
        for card in choice['not_picked']:
            cft_row = {
                'play_id':cft['play_id'][cft_index],
                'character_chosen':cft['character_chosen'][cft_index],
                'Card_Name':card,
                'floor':choice['floor'],
                'victory':cft['victory'][cft_index],
                'Card_Picked_Skipped':'Card Skipped'
                }
            cft_dictionary = pd.DataFrame([cft_row])
            cft_df = pd.concat([cft_df, cft_dictionary], ignore_index=True)
        cft_row = {
            'play_id':cft['play_id'][cft_index],
            'character_chosen':cft['character_chosen'][cft_index],
            'Card_Name':choice.get('picked'),
            'floor':choice['floor'],
            'victory':cft['victory'][cft_index],
            'Card_Picked_Skipped':'Card Picked'
            }
        cft_dictionary = pd.DataFrame([cft_row])
        cft_df = pd.concat([cft_df, cft_dictionary], ignore_index=True)
    cft_index +=1


cft_df[['floor','victory']] = cft_df[['floor','victory']].astype(int)
cft_df.to_csv("Cards for Tableau.csv", index = False)
cft_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5504 entries, 0 to 5503
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   play_id              5504 non-null   object
 1   character_chosen     5504 non-null   object
 2   Card_Name            5504 non-null   object
 3   floor                5504 non-null   int32 
 4   victory              5504 non-null   int32 
 5   Card_Picked_Skipped  5504 non-null   object
dtypes: int32(2), object(4)
memory usage: 215.1+ KB
